In [ ]:
import dbnl
import json
import pandas as pd
import pyarrow.parquet as pq
from datetime import UTC, datetime, timedelta

# Make sure your version matches the docs at https://docs.dbnl.com/
print("dbnl version:", dbnl.__version__)

In [ ]:
# Login to DBNL (using default Sandbox url)
dbnl.login(
    api_url="http://localhost:8080/api",
    api_token="<DBNL_API_KEY>", # found at http://localhost:8080/tokens
)

In [ ]:
# Create a new project
project = dbnl.get_or_create_project(
    name="ADK Calc Traces SDK via JSON",
    schedule="daily",  # How often DBNL analyzes new data
    default_llm_model_name="quickstart_model" # From step (2) in https://docs.dbnl.com/get-started/quickstart
)

In [ ]:
import pandas as pd

# Load the JSONL file
df = pd.read_json('traces.jsonl', lines=True)

# Convert timestamp columns to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# For nested timestamps in spans, we also need to convert to datetime
def convert_span_times(spans):
    for span in spans:
        span['start_time'] = pd.to_datetime(span['start_time'])
        span['end_time'] = pd.to_datetime(span['end_time'])
    return spans

df['spans'] = df['spans'].apply(convert_span_times)

In [ ]:
# if the data is spread across multiple days we need to break it into days for upload
df["date"] = df["timestamp"].dt.tz_convert("UTC").dt.date
day_dfs = [g.drop(columns="date").reset_index(drop=True)
           for _, g in df.groupby("date")]

print("Uploading data...")
print(f"See status at: http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}/status")

for idx, day_df in enumerate(day_dfs):
    print(f"{idx + 1} / {len(day_dfs)} publishing log data for {min(day_df['timestamp']).date()}")
    data_start_t = min(day_df['timestamp']).replace(hour=0, minute=0, second=0, microsecond=0)
    data_end_t = data_start_t + timedelta(days=1)
    try:
        dbnl.log(
            project_id=project.id,
            data_start_time=data_start_t,
            data_end_time=data_end_t,
            data=day_df,
        )
    except Exception as e:
        if "Data already exists" in str(e):
            continue
        raise

print("You can now explore your data in DBNL!")
print(f"http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}")